In [49]:
Embedding_Model = "hkunlp/instructor-xl"

In [50]:
LLM_Model = "google/flan-t5-large"

In [4]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


In [51]:
!pip install pymupdf

import fitz  # PyMuPDF
from langchain.schema import Document

def load_pdf_with_pymupdf(file_path):
  """Loads a PDF file using PyMuPDF and returns a langchain Document."""

  doc = fitz.open(file_path)  # Open the PDF file
  text = ""

  for page in doc:
    text += page.get_text()  # Extract text from each page

  return Document(page_content=text, metadata={"source": file_path})

# Replace with the actual path to your PDF file
file_path = "/content/pdf_files/AI_CLIMATE_CHANGE.pdf"
documents = load_pdf_with_pymupdf(file_path)

#print(document.page_content) # Print the extracted content

In [52]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [53]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# Wrap the 'documents' variable in a list
texts = text_splitter.split_documents([documents])

In [54]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [27]:
!pip install InstructorEmbedding

In [55]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings

instructor_embeddings = HuggingFaceEmbeddings(
    model_name="hkunlp/instructor-xl"
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [56]:
text = "This is a test document."
query_result = instructor_embeddings.embed_query(text)

In [35]:
query_result

[0.018697552382946014,
 -0.012886873446404934,
 0.009923241101205349,
 -0.026372626423835754,
 -0.012731232680380344,
 -0.020375162363052368,
 -0.08059597760438919,
 0.017575161531567574,
 -0.009408998303115368,
 -0.031238975003361702,
 0.025959480553865433,
 0.07744677364826202,
 -0.01530388556420803,
 -0.12628912925720215,
 -0.05694251134991646,
 0.01756621152162552,
 -0.008402287028729916,
 -0.09273533523082733,
 -0.0073073869571089745,
 0.003936609253287315,
 -0.030720794573426247,
 0.029214177280664444,
 -0.005141329951584339,
 -0.04588289186358452,
 -0.033559393137693405,
 -0.09744473546743393,
 0.0017913461197167635,
 0.05986844003200531,
 0.00795910693705082,
 0.002602623775601387,
 0.07920786738395691,
 -0.01852225326001644,
 -0.00286091398447752,
 -0.04705152288079262,
 0.01941615343093872,
 0.014966960996389389,
 -0.011247501708567142,
 -0.033110395073890686,
 0.007407630328088999,
 0.018797514960169792,
 -0.018361754715442657,
 0.05606565997004509,
 0.017432810738682747,
 -

In [57]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [58]:
tokenizer = AutoTokenizer.from_pretrained(LLM_Model)
model = AutoModelForSeq2SeqLM.from_pretrained(LLM_Model, torch_dtype=torch.float32)

In [59]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15
)

llm = HuggingFacePipeline(pipeline=pipe)

In [39]:
from langchain.vectorstores import Chroma

In [41]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.7/615.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.9 MB/s eta 0:00

In [42]:
persist_directory = 'db'
db = Chroma.from_documents(documents=texts,
                                 embedding=instructor_embeddings,
                                 persist_directory=persist_directory)

In [43]:
#retriever = db.as_retriever()
retriever = db.as_retriever(search_kwargs={"k": 3})

In [44]:
from langchain.chains import RetrievalQA

In [45]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [46]:
question = "solutions to Climate Change?"
generated_text = qa(question)
generated_text

<ipython-input-46-246a7cdf01e6>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  generated_text = qa(question)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


{'query': 'solutions to Climate Change?',
 'result': 'Switching energy systems from fossil fuels to renewables',
 'source_documents': [Document(metadata={'source': '/content/pdf_files/fastfacts-what-is-climate-change.pdf'}, page_content='10. Switching energy systems from fossil fuels to renewables like solar will reduce \nthe emissions driving climate change. But we have to start right now. While a growing \ncoalition of countries is committing to net zero emissions by 2050, about half of \nemissions cuts must be in place by 2030 to keep warming below 1.5°C. Fossil fuel \nproduction must decline by roughly 6 per cent per year between 2020 and 2030.'),
  Document(metadata={'source': '/content/pdf_files/fastfacts-what-is-climate-change.pdf'}, page_content='creating more of the problem have a greater responsibility to act first.   \n9. Climate change is a huge challenge, but we already know many solutions. These can \ndeliver economic benefits while improving our lives and protecting the 

In [60]:
question = "what are the greenhouse gases that make climate changes?"
generated_text = qa(question)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [61]:
generated_text

{'query': 'what are the greenhouse gases that make climate changes?',
 'result': 'carbon dioxide and methane',
 'source_documents': [Document(metadata={'source': '/content/pdf_files/fastfacts-what-is-climate-change.pdf'}, page_content='wrapped around the earth, trapping the sun’s heat and raising temperatures.\n3. Examples of greenhouse gas emissions that are causing climate change include \ncarbon dioxide and methane. These come from burning fossil fuels such as gasoline \nfor driving a car or coal for heating a building. Clearing land and forests can also \nrelease carbon dioxide. Landfills for garbage are another source. Energy, industry, \nagriculture and waste disposal are among the major emitters.'),
  Document(metadata={'source': '/content/pdf_files/fastfacts-what-is-climate-change.pdf'}, page_content='What Is Climate Change?\n1. Climate change can be a natural process where temperature, rainfall, wind and \nother elements vary over decades or more. In millions of years, our wor